In [ ]:
import pandas as pd

# Load the dataset
try:
    df = pd.read_csv('final_food_delivery_dataset.csv')
except:
    # Fallback to regeneration if file is missing (robustness)
    orders_df = pd.read_csv('orders.csv')
    users_df = pd.read_json('users.json')
    import re
    restaurant_data = []
    with open('restaurants.sql', 'r') as f:
        for line in f:
            if line.strip().startswith("INSERT INTO restaurants VALUES"):
                match = re.search(r"\((.*)\);", line)
                if match:
                    restaurant_data.append([p.strip().strip("'") for p in match.group(1).split(',')])
    restaurants_df = pd.DataFrame(restaurant_data, columns=['restaurant_id', 'restaurant_name', 'cuisine', 'rating'])
    restaurants_df['restaurant_id'] = pd.to_numeric(restaurants_df['restaurant_id'])
    
    merged = pd.merge(orders_df, users_df, on='user_id', how='left')
    df = pd.merge(merged, restaurants_df, on='restaurant_id', how='left', suffixes=('_order', '_details'))

# Group by cuisine and calculate average order value
cuisine_avg_order_value = df.groupby('cuisine')['total_amount'].mean().sort_values(ascending=False)

print("Average Order Value by Cuisine:")
print(cuisine_avg_order_value)
print("\nTop Cuisine:", cuisine_avg_order_value.index[0])
print("Highest Average Order Value:", cuisine_avg_order_value.iloc[0])